In [57]:
import requests
from bs4 import BeautifulSoup
import re

def extract_int_from_string(input_string):
    # Use a regular expression to find the number in the string
    match = re.search(r'\d[\d,]*', input_string)
    if match:
        # Remove commas and convert to integer
        number = int(match.group().replace(',', ''))
        return number
    return None


def name_to_id(artist_name):
    artist_dict = {"Azahriah":"6EIriUxo7vznEgJtTDlXpq?si=iThY86mXTe28gVdZy7jNTA"}
    try:
        artist_id  = artist_dict[artist_name]
        return artist_id
    except KeyError:
        raise KeyError(f"Artist '{artist_name}' not in database")



def get_spotify_listeners(artist_name):
    
    artist_id = name_to_id(artist_name)
    
    # Get artist page from Spotify
    artist_url = f"https://open.spotify.com/artist/{artist_id}"
    response = requests.get(artist_url)
    
    # Check if the request was successful
    if response.status_code != 200:
        print(f"Failed to fetch artist page for {artist_name}. Status code: {response.status_code}")
    
    # Parse the artist's page
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the monthly listeners
    monthly_listeners = soup.find('div', {'data-testid': 'monthly-listeners-label'}).text
    
    monthly_listeners = extract_int_from_string(monthly_listeners)
    
    if not monthly_listeners:
        print(f"Monthly listeners not found for {artist_name}.")
        return None
    
    return monthly_listeners

In [82]:
import json
import pandas as pd
from get_monthly_listeners import get_spotify_listeners
from datetime import datetime

def add_listeners_to_csv(json_file='artist_db.json', csv_file='listeners_data.csv'):
    # Load the dictionary from the JSON file
    with open(json_file, 'r') as file:
        artist_dict = json.load(file)
    
    # Read the existing CSV file into a DataFrame
    try:
        df = pd.read_csv(csv_file)
    except FileNotFoundError:
        # If the file does not exist, create an empty DataFrame with the correct columns
        df = pd.DataFrame(columns=['Date'] + list(artist_dict.keys()))

    
    current_date = datetime.now().strftime('%Y-%m-%d')

    # Check if the current date is already in the DataFrame
    if current_date in df['Date'].values:
        if set(df.columns) == set(list(artist_dict.keys()) + ['Date']):
            print("Stock data up to date")
            return
        
        df = df[df['Date'] != current_date]
    
        # Check for missing artist columns and add them with NaN values
        for artist_name in artist_dict.keys():
            if artist_name not in df.columns:
                print(f"Adding new artist {artist_name} to data and resetting today's values")
                df[artist_name] = pd.Series([float('nan')] * len(df))
    
    # Create a dictionary to hold the new row of data
    new_row = {'Date': current_date}

    # Iterate through the dictionary keys
    for artist_name in artist_dict.keys():
        # Call the get_spotify_listeners function and store the result
        listeners = get_spotify_listeners(artist_name)
        new_row[artist_name] = listeners
    
    # Append the new row to the DataFrame
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
    

    # Save the updated DataFrame to the CSV file
    df.to_csv(csv_file, index=False)

def main():
    add_listeners_to_csv()

if __name__ == '__main__':
    main()

Adding new artist Moriones to data and resetting today's values


In [83]:
df = pd.read_csv("listeners_data.csv")
df.head()

,Date,Azahriah,Pogány Induló,Elefánt,Moriones
0,2024-07-30,700135,356819,114427,203364


In [64]:
import json
import pandas as pd
from get_monthly_listeners import get_spotify_listeners
from datetime import datetime

def add_listeners_to_csv(json_file='artist_db.json', csv_file='listeners_data.csv'):
    # Load the dictionary from the JSON file
    with open(json_file, 'r') as file:
        artist_dict = json.load(file)
    
    # Read the existing CSV file into a DataFrame
    try:
        df = pd.read_csv(csv_file)
    except FileNotFoundError:
        # If the file does not exist, create an empty DataFrame with the correct columns
        columns = ['Date'] + list(artist_dict.keys())
        df = pd.DataFrame(columns=columns)
        # Initialize with a row that has NaN for each column
        df = df.append(pd.Series([float('nan')] * len(columns), index=columns), ignore_index=True)

    # Check for missing artist columns and add them with NaN values
    for artist_name in artist_dict.keys():
        if artist_name not in df.columns:
            df[artist_name] = pd.Series([float('nan')] * len(df))

    # Create a dictionary to hold the new row of data
    new_row = {'Date': datetime.now().strftime('%Y-%m-%d')}

    # Iterate through the dictionary keys
    for artist_name in artist_dict.keys():
        # Call the get_spotify_listeners function and store the result
        listeners = get_spotify_listeners(artist_name)
        new_row[artist_name] = listeners
    
    # Append the new row to the DataFrame
    df = df.append(new_row, ignore_index=True)

    # Save the updated DataFrame to the CSV file
    df.to_csv(csv_file, index=False)

# Example usage
add_listeners_to_csv()

AttributeError: 'DataFrame' object has no attribute 'append'

In [17]:
import json
import pandas as pd

# File paths
users_file = 'users.json'
listeners_file = './data/listeners_data.csv'

# Load JSON data
def load_users():
    try:
        with open(users_file, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        return {}

# Save JSON data
def save_users(users):
    with open(users_file, 'w') as f:
        json.dump(users, f, indent=4)

# Add user
def add_user(username):
    users = load_users()
    if username not in users:
        users[username] = {
            'Username': username,
            'Stocks': [],
            'FloatingMoney': 100,
            'PortfolioValue': 100
        }
        save_users(users)
        print(f"User {username} added successfully.")
    else:
        print(f"User {username} already exists.")

# Calculate Portfolio
def calculate_portfolio(username):
    users = load_users()
    if username in users:
        df = pd.read_csv(listeners_file)
        last_row = df.iloc[-1]
        portfolio_value = 0
        
        for stock in users[username]['Stocks']:
            artist_name = stock['ArtistName']
            current_gross = last_row[artist_name]
            stock['CurrentValue'] = (current_gross / stock['GrossAtPurchase']) * stock['AmountPurchased']
            portfolio_value += stock['CurrentValue']
        
        users[username]['PortfolioValue'] = portfolio_value
        save_users(users)
        print(f"Portfolio for {username} updated successfully.")
    else:
        print(f"User {username} not found.")

# Buy stock
def buy_stock(username, artist_name, amount_purchased):
    users = load_users()
    if username in users:
        df = pd.read_csv(listeners_file)
        last_row = df.iloc[-1]
        
        if users[username]['FloatingMoney'] >= amount_purchased:
            current_gross = last_row[artist_name]
            stock_exists = False
            
            for stock in users[username]['Stocks']:
                if stock['ArtistName'] == artist_name:
                    stock['AmountPurchased'] += amount_purchased
                    stock['GrossAtPurchase'] = current_gross
                    stock_exists = True
                    break
            
            if not stock_exists:
                new_stock = {
                    'ArtistName': artist_name,
                    'CurrentValue': amount_purchased,  # Will be updated in calculate_portfolio
                    'AmountPurchased': amount_purchased,
                    'GrossAtPurchase': int(current_gross)
                }
                users[username]['Stocks'].append(new_stock)
            
            users[username]['FloatingMoney'] -= amount_purchased
            calculate_portfolio(username)
            save_users(users)
            print(f"Stock {artist_name} purchased successfully for user {username}.")
        else:
            print("Insufficient funds.")
    else:
        print(f"User {username} not found.")

In [18]:
# Example usage
add_user('johndoe')

User johndoe added successfully.


In [20]:
buy_stock('johndoe', 'Elefánt', 10)

Portfolio for johndoe updated successfully.
Stock Elefánt purchased successfully for user johndoe.


In [ ]:
calculate_portfolio('johndoe')